In [1]:
# 變數前處理

%run preprocessing.py
print ("shape of usersXprod :", usersXprod.shape)

p = pd.read_csv("products.csv")
p = p.drop(["product_name"], axis=1)
usersXprod = pd.merge(usersXprod, p, how='left', on=['product_id'])


import data
merge orders and order_products__prior
size of prd features : 5
size of users features : 13
nb of usersXproducts features : 13
merge prod, user and usersXprod feature on usersXprod
order row 2000000
order row 4000000
order row 6000000
order row 8000000
order row 10000000
order row 12000000
shape of usersXprod : (13307953, 32)


In [2]:
import time

class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))

# convert to form of submit
def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
    '''Create statistical columns, group by [N columns] and compute stats on [1 column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       target_columns_list: list_like
          column you want to compute stats, need to be a list with only one element
       methods_list: list_like
          methods that you want to use, all methods that supported by groupby in Pandas

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       ka_add_stats_features_n_vs_1(train, group_columns_list=['x0'], target_columns_list=['x10'])
    '''
    with tick_tock("add stats features", verbose):
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new


In [3]:
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score

train = usersXprod.loc[usersXprod.eval_set == "train",:]
train.drop(['eval_set', 'user_id', 'product_id', 'order_id'], axis=1, inplace=True)

X_test = usersXprod.loc[usersXprod.eval_set == "test",:]

X_train, X_val, y_train, y_val = train_test_split(train.drop('reordered', axis=1),
                                                  train.reordered,
                                                  test_size = 0.99,
                                                  random_state = 42)


/Users/Shawn/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
# Choose classifier
clf = xgboost.XGBClassifier(verbose = 100)

# Parameters to try
xgb_params = {
    "objective"        : ["reg:logistic"],
    'learning_rate'    : [0.05, 0.1],
    "max_depth"        : [6, 8],
    "min_child_weight" : [10],
    "gamma"            : [0.70],
    "subsample"        : [0.76],
    "n_estimators"     : [300, 1000], #number of trees, change it to 1000 for better results          
    "colsample_bytree" : [0.95]
}

# Run the grid search
grid_obj = GridSearchCV(clf, xgb_params, scoring = 'roc_auc', verbose = 100)
grid_obj = grid_obj.fit(X_train, y_train)

clf = grid_obj.best_estimator_
print(clf)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.76 
[CV]  colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.76, score=0.822501, total=  29.1s
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.76 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.8s remaining:    0.0s


[CV]  colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.76, score=0.825677, total=  20.4s
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.76 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   50.7s remaining:    0.0s


[CV]  colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.76, score=0.828048, total=  20.5s
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, n_estimators=1000, objective=reg:logistic, subsample=0.76 


...

[CV]  colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=8, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.76, score=0.824074, total=  35.9s
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=8, min_child_weight=10, n_estimators=1000, objective=reg:logistic, subsample=0.76 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  7.5min remaining:    0.0s


[CV]  colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=8, min_child_weight=10, n_estimators=1000, objective=reg:logistic, subsample=0.76, score=0.802900, total= 1.9min
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=8, min_child_weight=10, n_estimators=1000, objective=reg:logistic, subsample=0.76 
[CV]  colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=8, min_child_weight=10, n_estimators=1000, objective=reg:logistic, subsample=0.76, score=0.807079, total= 2.0min
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=8, min_child_weight=10, n_estimators=1000, objective=reg:logistic, subsample=0.76 
[CV]  colsample_bytree=0.95, gamma=0.7, learning_rate=0.05, max_depth=8, min_child_weight=10, n_estimators=1000, objective=reg:logistic, subsample=0.76, score=0.809041, total= 1.9min
[CV] colsample_bytree=0.95, gamma=0.7, learning_rate=0.1, max_depth=6, min_child_weight=10, n_estimators=300, objective=reg:logistic, subsample=0.7

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 27.4min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.95,
       gamma=0.7, learning_rate=0.05, max_delta_step=0, max_depth=6,
       min_child_weight=10, missing=None, n_estimators=300, nthread=-1,
       objective='reg:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.76)


In [5]:
# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

predictions = clf.predict(X_val)
print(accuracy_score(y_val, predictions))

0.909264158219


In [6]:
from sklearn.ensemble import BaggingClassifier as bc
bc = bc(base_estimator     = clf,
        n_estimators       = 10,
        max_samples        = 1.0,
        max_features       = 1.0,
        bootstrap          = True,
        bootstrap_features = True,
        oob_score          = False,
        warm_start         = False,
        n_jobs             = 1,
        random_state       = 2,
        verbose            = 10)

clf.fit(train.drop('reordered', axis=1), train.reordered)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.95,
       gamma=0.7, learning_rate=0.05, max_delta_step=0, max_depth=6,
       min_child_weight=10, missing=None, n_estimators=300, nthread=-1,
       objective='reg:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.76)

In [7]:
# predict
# convert to the form of submit

d_test = X_test.drop(['eval_set', 'user_id', 'order_id', 'reordered', 'product_id'], axis=1)
X_test.loc[:, 'reordered'] = (clf.predict(d_test) > 0.21).astype(int)
X_test.loc[:, 'product_id'] = X_test.product_id.astype(str)

submit = ka_add_groupby_features_n_vs_1(X_test[X_test.reordered == 1], 
                                        group_columns_list = ['order_id'],
                                        target_columns_list = ['product_id'],
                                        methods_list = [lambda x: ' '.join(set(x))],
                                        keep_only_stats = True)
sample_submission = pd.read_csv("sample_submission.csv")
print (len(sample_submission), len(submit))

submit.columns = sample_submission.columns.tolist()
submit_final = sample_submission[['order_id']].merge(submit, how='left').fillna('None')
submit_final.to_csv("submit_xgboost_GridSearchCV.csv", index = False)


add stats features begin ......
add stats features end ......
time lapsing 2.597749948501587 s 

75000 43492
